# All the parameters that need to be changed

In [1]:
# Kansas
state_ab = "ks"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [2]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/M3_AdAstra_2 for KLRD TR.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/Liberty_3 for KLRD TR.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/Freestate 3F for KLRD TR.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|██████████████████████████████████████| 4062/4062 [00:08<00:00, 501.85it/s]


There are 1289 overlaps.
There are 9674 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 6644/6644 [00:02<00:00, 2835.29it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 1201it [23:41,  1.18s/it]                                     
100%|██████████████████████████████████████| 4062/4062 [00:04<00:00, 823.95it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [19]:
vest20.columns

Index(['COUNTYFP', 'VTDST', 'NAMELSAD', 'G20PREDBID', 'G20PRERTRU',
       'G20PRELJOR', 'G20USSDBOL', 'G20USSRMAR', 'G20USSLBUC', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [20]:
start_col = 3
vest_base_data = vest20
year = '20'

In [21]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [22]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|███████████████████████████████████████| 4062/4062 [00:41<00:00, 98.30it/s]


In [23]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [24]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [25]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [26]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [27]:
vest18 = gpd.read_file(vest18_data)

In [28]:
vest18.columns

Index(['COUNTYFP', 'VTDST', 'NAMELSAD', 'G18GOVDKEL', 'G18GOVRKOB',
       'G18GOVIORM', 'G18GOVLCAL', 'G18GOVIKLO', 'G18SOSRSCH', 'G18SOSDMCC',
       'G18SOSLHOD', 'G18ATGRSCH', 'G18ATGDSWA', 'G18INSDMCL', 'G18INSRSCH',
       'G18TRERLAT', 'G18TREDFRA', 'geometry'],
      dtype='object')

In [29]:
# check the result here
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|██████████████████████████████████████| 4051/4051 [00:07<00:00, 523.51it/s]


There are 1137 overlaps.
There are 7362 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 6220/6220 [00:01<00:00, 3113.58it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 1019it [21:05,  1.24s/it]                                     
100%|██████████████████████████████████████| 4062/4062 [00:38<00:00, 104.81it/s]


population agrees


In [30]:
vest16 = gpd.read_file(vest16_data)
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|██████████████████████████████████████| 4021/4021 [00:06<00:00, 584.43it/s]


There are 1323 overlaps.
There are 11091 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 6214/6214 [00:01<00:00, 3133.89it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 1023it [21:13,  1.25s/it]                                     
100%|██████████████████████████████████████| 4062/4062 [00:37<00:00, 108.07it/s]


population agrees


## Add the district data

In [31]:
cong_df = gpd.read_file(cd_data)
send = gpd.read_file(send_data)
hdist = gpd.read_file(hdist_data)

In [32]:
cong_df

,ID,AREA,DATA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,...,F_NH18_OTH,F_NH18_2_R,F_NH18_DOJ,F_NH18_DO1,F_NH18_DO2,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,DISTRICT_L,geometry
0,1,14630.037109,4,4,1.0,None,None,734470,538330,50124,...,0.002334,0.033813,0.052942,0.023621,0.028845,0.001023,0.007322,0.003852,4|0%,"POLYGON ((-98.91258 38.26109, -98.91246 38.261..."
1,2,49802.121094,1,1,1.0,None,None,734470,579523,20476,...,0.002325,0.025802,0.024857,0.018484,0.017643,0.000767,0.007343,0.002298,1|0%,"POLYGON ((-99.54112 36.99961, -99.54157 36.999..."
2,3,15494.653320,2,2,1.0,None,None,734470,536135,63310,...,0.002287,0.033545,0.068788,0.024427,0.015332,0.001567,0.007433,0.003808,2|0%,"POLYGON ((-96.03595 39.56610, -96.03634 39.566..."
3,4,2292.086426,3,3,1.0,None,None,734470,568474,34899,...,0.002554,0.028442,0.037540,0.014473,0.039574,0.000705,0.008632,0.002285,3|0%,"POLYGON ((-95.05626 38.98212, -95.05582 38.981..."


In [33]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 4/4 [00:06<00:00,  1.74s/it]


In [34]:
send.head()

,ID,AREA,DATA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,...,F_NH18_OTH,F_NH18_2_R,F_NH18_DOJ,F_NH18_DO1,F_NH18_DO2,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,DISTRICT_L,geometry
0,1,8811.151360,33,39,1.0,None,None,72519,43311,1679,...,0.002041,0.016644,0.016327,0.012121,0.017030,0.000331,0.005764,0.001737,39|-1.26%,"POLYGON ((-100.22714 38.26224, -100.22707 38.2..."
1,2,12996.666900,35,40,1.0,None,None,72101,64437,876,...,0.001942,0.021622,0.012330,0.011082,0.006019,0.000374,0.007698,0.002954,40|-1.83%,"POLYGON ((-100.24720 38.69817, -100.25044 38.6..."
2,3,6000.474600,32,38,1.0,None,None,73453,40473,1739,...,0.001239,0.012607,0.017658,0.008699,0.011599,0.000123,0.003621,0.001457,38|0.01%,"POLYGON ((-99.54112 36.99961, -99.54157 36.999..."
3,4,3273.222160,28,34,1.0,None,None,73596,63543,1909,...,0.001834,0.027814,0.024186,0.020667,0.005788,0.000720,0.006712,0.001400,34|0.2%,"POLYGON ((-98.34715 36.99815, -98.34802 36.998..."
4,5,480.567993,19,26,1.0,None,None,73190,61536,1243,...,0.001886,0.031589,0.014906,0.022626,0.017885,0.000724,0.007323,0.002282,26|-0.35%,"POLYGON ((-97.80760 37.47418, -97.80758 37.475..."


In [35]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 40/40 [00:04<00:00,  8.07it/s]


In [36]:
hdist.head()

,ID,AREA,DATA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,...,F_NH18_OTH,F_NH18_2_R,F_NH18_DOJ,F_NH18_DO1,F_NH18_DO2,F_NH18_DO3,F_NH18_DO4,F_NH18_DO5,DISTRICT_L,geometry
0,1,6.174718,109,84,1.0,None,None,23539,9738,7622,...,0.002974,0.039594,0.240410,0.015719,0.025320,0.001147,0.005268,0.013297,84|0.15%,"POLYGON ((-97.31826 37.72266, -97.31817 37.722..."
1,2,13.106968,114,89,1.0,None,None,24153,11290,7585,...,0.003643,0.033826,0.243200,0.014532,0.067362,0.000580,0.006459,0.009771,89|2.77%,"POLYGON ((-97.26248 37.76883, -97.26248 37.768..."
2,3,49.567436,107,82,1.0,None,None,23223,20067,303,...,0.001636,0.035267,0.011626,0.027559,0.013693,0.001033,0.006158,0.002067,82|-1.19%,"POLYGON ((-97.15333 37.47554, -97.15329 37.474..."
3,4,40.998199,106,81,1.0,None,None,23040,15471,1689,...,0.002387,0.035547,0.055903,0.021267,0.046745,0.002170,0.008377,0.004687,81|-1.97%,"POLYGON ((-97.15282 37.57762, -97.15282 37.574..."
4,5,15.171526,113,88,1.0,None,None,23527,12273,3682,...,0.002423,0.038381,0.121052,0.017257,0.125260,0.001785,0.006036,0.007268,88|0.1%,"POLYGON ((-97.13804 37.63597, -97.13994 37.635..."


In [37]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|█████████████████████████████████████████| 125/125 [00:05<00:00, 22.59it/s]


In [38]:
election_df.columns

Index(['2MOREVAP', 'AMINVAP', 'ASIANVAP', 'ATG18D', 'ATG18R', 'BVAP',
       'COUNTYFP', 'GOV18D', 'GOV18O', 'GOV18R', 'HISP', 'HVAP', 'H_2MORE',
       'H_AMIN', 'H_ASIAN', 'H_BLACK', 'H_NHPI', 'H_OTHER', 'H_WHITE',
       'INS18D', 'INS18R', 'NAMELSAD', 'NHPIVAP', 'NH_2MORE', 'NH_AMIN',
       'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER', 'NH_WHITE', 'OTHERVAP',
       'PRE16O', 'PRE16R', 'PRE20D', 'PRE20O', 'PRE20R', 'SOS18D', 'SOS18O',
       'SOS18R', 'TOTPOP', 'TRE18D', 'TRE18R', 'USS16D', 'USS16O', 'USS16R',
       'USS20D', 'USS20O', 'USS20R', 'VAP', 'VTDST', 'WVAP', 'geometry', 'CD',
       'SEND', 'HDIST'],
      dtype='object')

### Put the base precinct year after the precinct information column

In [39]:
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'VTDST':'VTDST'+year,
        'GEOID':'GEOID'+year,
        'NAME':'NAME'+year,
        'NAMELSAD':'NAMELSAD'+year
    }
election_df.rename(columns=base_columns, inplace = True)

In [40]:
election_df.columns

Index(['2MOREVAP', 'AMINVAP', 'ASIANVAP', 'ATG18D', 'ATG18R', 'BVAP',
       'COUNTYFP20', 'GOV18D', 'GOV18O', 'GOV18R', 'HISP', 'HVAP', 'H_2MORE',
       'H_AMIN', 'H_ASIAN', 'H_BLACK', 'H_NHPI', 'H_OTHER', 'H_WHITE',
       'INS18D', 'INS18R', 'NAMELSAD', 'NHPIVAP', 'NH_2MORE', 'NH_AMIN',
       'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER', 'NH_WHITE', 'OTHERVAP',
       'PRE16O', 'PRE16R', 'PRE20D', 'PRE20O', 'PRE20R', 'SOS18D', 'SOS18O',
       'SOS18R', 'TOTPOP', 'TRE18D', 'TRE18R', 'USS16D', 'USS16O', 'USS16R',
       'USS20D', 'USS20O', 'USS20R', 'VAP', 'VTDST20', 'WVAP', 'geometry',
       'CD', 'SEND', 'HDIST'],
      dtype='object')

In [46]:
election_df = election_df.rename(columns={'NAMELSAD':'NAMELSAD'+year})

In [47]:
# reorder the columns
fixed_columns = [
    'COUNTYFP'+year,
    'VTDST'+year,
    'NAMELSAD'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [48]:
list(election_df.columns)

['COUNTYFP20',
 'VTDST20',
 'NAMELSAD20',
 'CD',
 'SEND',
 'HDIST',
 'TOTPOP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'HISP',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'H_2MORE',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'ATG18D',
 'ATG18R',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'INS18D',
 'INS18R',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'TRE18D',
 'TRE18R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS20D',
 'USS20O',
 'USS20R',
 'geometry']

In [49]:
# store the result in directory "ks"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))